In [9]:
import pickle
import numpy as np
ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']
ACTION_TO_INT = {'UP':0, 'RIGHT':1, 'DOWN':2, 'LEFT':3, 'WAIT':4, 'BOMB':5}

In [10]:
# TODO: check if the linear value approximation actually works, by approximating only for one case
# TODO: check out regression forests
# TODO: check put PyTables to save and load entries of large dictionaries saved as a file

In [15]:
batch = [ [np.array([ 0,  1,  1,  1,  1, -1]), np.array([-164.52899074])] ]
beta = np.zeros(6)
LIN_VAL_TRAINING_RATE = 0.05
N = 1

In [16]:


s = 0 # sum
for (X, Y) in batch:
    s += X.T * ( Y - np.dot(X, beta) )
new_beta = beta + (LIN_VAL_TRAINING_RATE / N) * s

print("new_beta: ", new_beta)
print()


array([ 0.        , -8.22644954, -8.22644954, -8.22644954, -8.22644954,
        8.22644954])

In [6438]:
ACTIONS[ACTION_TO_INT['BOMB']]

'BOMB'

In [6439]:
with open("my-saved-q-table.pt", "rb") as file:
    q_table:dict = dict(pickle.load(file))
with open("action_vectors.pt", "rb") as file:
    action_vectors:dict = dict(pickle.load(file))

In [6440]:
mystr = "[ 0  1  0 -2]"
mystr = mystr[1:-1]
#print (np.array(list(mystr)))

test = np.fromstring(mystr, dtype=float, sep=' ')
type(test)

numpy.ndarray

In [6441]:
def state_to_action_via_lin_val_est(state_str:str):
    temp_str = state_str[1:-1]
    state = np.fromstring(temp_str, dtype=float, sep=' ')
    Q = np.zeros(len(ACTIONS))
    for i, a in enumerate(ACTIONS):
        Q[i] = np.dot(state, action_vectors[a])
    action_int = Q.argmax()
    return ACTIONS[action_int]

In [6442]:
c = 0
for key in sorted(q_table.keys()):
    #print(key)
    action = q_table[key].argmax()
    print(key, "simple_q_learning: ", ACTIONS[action], "lin_val_approx: ", state_to_action_via_lin_val_est(key) )
    if (ACTIONS[action] ==state_to_action_via_lin_val_est(key)):
        c += 1
    print(q_table[key] )
    print()

[ 0  1  1  1  1 -1] simple_q_learning:  RIGHT lin_val_approx:  LEFT
[-164.52899074  -36.76160623 -113.90732006 -100.78677762 -181.33350842
 -195.79401605]

[ 0  1  1  2  1 -1] simple_q_learning:  LEFT lin_val_approx:  LEFT
[-198.89910737 -209.34945938 -298.94504885 -135.67157266 -265.42607637
 -236.25486353]

[ 0  1  1  3  1 -1] simple_q_learning:  RIGHT lin_val_approx:  LEFT
[-113.25044067    8.3102088   -81.75225019  -67.15097274 -124.17896386
  -91.78516893]

[ 0  1  1 -1  1 -1] simple_q_learning:  LEFT lin_val_approx:  LEFT
[-249.33147776 -230.43585519 -262.21787079 -107.27228884 -204.16372052
 -214.87773547]

[ 0  1  1 -2  1 -1] simple_q_learning:  LEFT lin_val_approx:  LEFT
[-196.33271301 -155.26712604 -215.80262902 -110.25130517 -155.06031108
 -207.87376588]

[ 0  1  2  1  1 -1] simple_q_learning:  RIGHT lin_val_approx:  LEFT
[-173.4878898   -22.5686193  -189.6066073  -147.48683753 -194.44446391
 -181.00695994]

[ 0  1  2  2  1 -1] simple_q_learning:  RIGHT lin_val_approx:  LEFT

In [6443]:
print(len(q_table)) 
print(c)

66
41


In [6444]:

for key in action_vectors:
    print(key, action_vectors[key])

UP [ 56.60790713 -30.88760512  24.02032043   2.88909789 -82.09983692
  83.38596739]
RIGHT [-13.48795983  15.59505074  16.32011977  19.10109073 -82.72808019
  83.47350781]
DOWN [ 25.11773113 -29.82734613  11.65146627  19.79966639 -79.4966889
  80.71568127]
LEFT [-32.73583404  48.72209702  -0.21881048  10.42588635 -73.15024407
  74.06858543]
WAIT [  0.33384657  -5.45218068  13.25818769   9.40329015 -95.14276654
  95.6877198 ]
BOMB [  14.6868379    11.51985639   16.74329043   11.84613422 -108.95828287
  110.21866247]


In [6445]:
v = np.array([-3.39690401e+001, -3.78378868e+001, -5.91096107e+254,  1.12225685e+227,
 -3.83210288e+001,  3.83210289e+001])
s = np.array([ 1,  1, 15, 16,  1, -1])
np.dot(s, v)

-8.866441605e+255

In [6446]:
training_batches = dict()
for a in ACTIONS:
    training_batches[a] = []

In [6447]:
def update_training_batches(n_max_samples:int = 10):
    # take randomly 10 samples from the q table and put the q-values in the according training_batches
    n_samples = min(len(q_table), n_max_samples)
    random_states_str = random.sample(list(q_table), n_samples)
    print(len(random_states_str))
    temp = [s[1:-1] for s in random_states_str]  # remove brackets from the strings
    features = [np.fromstring(s, dtype=float, sep=' ') for s in temp]   # convert strings back to numpy arrays

    for i, key in enumerate( random_states_str):
        for j, a in enumerate(ACTIONS):
            training_batches[a].append( [features[i], q_table[key][j] ])
    return training_batches

In [6448]:
test = update_training_batches()


10


In [6449]:
def dist_to_proximity(pos:tuple, max_dist:int = 15)->tuple:
    # converts relative position to proximity values (high values mean that the position is near)
    a = max_dist - np.abs(pos[0])
    b = max_dist - np.abs(pos[1])
    if pos[0] < 0:
        a *= -1
    if pos[1] < 0:
        b *= -1
    return (a, b)

In [6450]:
dist_to_proximity((2, 0), 3)

(1, 3)